<a href="https://colab.research.google.com/github/AbelRapha/DataProjects/blob/main/Projetos%20em%20Python/Analise_de_Munic%C3%ADpios_por_Georefer%C3%AAncia.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import io #Biblioteca que manipula outputs e inputs de formato string
import requests #Pega dados de outros servidores

url = " https://github.com/kelvins/Municipios-Brasileiros/blob/main/csv/municipios.csv"
s = requests.get(url).content
s

In [17]:
file_name = io.StringIO(s.decode("utf-8"))
print(file_name)
dados = pd.read_html(file_name)
df = dados[0] #Daata frame esta no indice 0 da lista

#Limpando os dados
df1 = df.copy()
df1.iloc[:,0] = df1.iloc[:,0].fillna(0) #preenche todos os valores da coluna Unnamed com 0
#Eliminando a coluna Unnamed
df2 = df.copy()
df2 = df2.transpose() #Transforma linha por coluna e vice-versa
df2 = df2.dropna() #Apaga todas as linhas que possuem NaN
df2 = df2.transpose()
df2

,codigo_ibge,nome,latitude,longitude,capital,codigo_uf,siafi_id,ddd,fuso_horario
0,5200050,Abadia de Goiás,-16.7573,-49.4412,0,52,1050,62,America/Sao_Paulo
1,3100104,Abadia dos Dourados,-18.4831,-47.3916,0,31,4001,34,America/Sao_Paulo
2,5200100,Abadiânia,-16.197,-48.7057,0,52,9201,62,America/Sao_Paulo
3,3100203,Abaeté,-19.1551,-45.4444,0,31,4003,37,America/Sao_Paulo
4,1500107,Abaetetuba,-1.72183,-48.8788,0,15,401,91,America/Sao_Paulo
...,...,...,...,...,...,...,...,...,...
5565,2933604,Xique-Xique,-10.823,-42.7245,0,29,3971,74,America/Sao_Paulo
5566,2517407,Zabelê,-8.07901,-37.1057,0,25,542,83,America/Sao_Paulo
5567,3557154,Zacarias,-21.0506,-50.0552,0,35,2973,18,America/Sao_Paulo
5568,2114007,Zé Doca,-3.27014,-45.6553,0,21,1287,98,America/Sao_Paulo


In [14]:
print(df2.columns.ravel()) #Mostra os nomes das colunas


['codigo_ibge' 'nome' 'latitude' 'longitude' 'capital' 'codigo_uf'
 'siafi_id' 'ddd' 'fuso_horario']


In [19]:
print(df2.loc[df['codigo_uf'] == 35])

     codigo_ibge                  nome latitude  ... siafi_id ddd       fuso_horario
30       3500105            Adamantina  -21.682  ...     6101  18  America/Sao_Paulo
32       3500204                Adolfo -21.2325  ...     6103  17  America/Sao_Paulo
62       3500303                 Aguaí -22.0572  ...     6105  19  America/Sao_Paulo
65       3500402        Águas da Prata -21.9319  ...     6107  19  America/Sao_Paulo
67       3500501      Águas de Lindóia -22.4733  ...     6109  19  America/Sao_Paulo
...          ...                   ...      ...  ...      ...  ..                ...
5528     3556909  Vista Alegre do Alto -21.1692  ...     7241  16  America/Sao_Paulo
5534     3556958        Vitória Brasil -20.1956  ...      828  17  America/Sao_Paulo
5545     3557006            Votorantim -23.5446  ...     7243  15  America/Sao_Paulo
5546     3557105           Votuporanga -20.4237  ...     7245  17  America/Sao_Paulo
5567     3557154              Zacarias -21.0506  ...     2973  18

In [22]:
df3 = df2.loc[df['codigo_uf'] == 35, ['nome', 'latitude', 'longitude']] #Pegando os municipios que possuem uf 35
df3

,nome,latitude,longitude
30,Adamantina,-21.682,-51.0737
32,Adolfo,-21.2325,-49.6451
62,Aguaí,-22.0572,-46.9735
65,Águas da Prata,-21.9319,-46.7176
67,Águas de Lindóia,-22.4733,-46.6314
...,...,...,...
5528,Vista Alegre do Alto,-21.1692,-48.6284
5534,Vitória Brasil,-20.1956,-50.4875
5545,Votorantim,-23.5446,-47.4388
5546,Votuporanga,-20.4237,-49.9781


In [31]:
#Criando o mapa
import folium
#Cria um mapa de acordo com a configuranções de geolocalização e zoom
map = folium.Map(location = [-22.7864889, -50.678708], zoom_start=7)
for i in range (0,len(df3)): #ira criar varias marcações de acordo com a latitude e longitude do df3
  pop = str(df3.iloc[i]['nome'])
  pop = pop.encode("latin1") #Configurancao de idioma
  folium.Marker([df3.iloc[i]['latitude'], df3.iloc[i]['longitude']], popup =pop).add_to(map)

map  

In [37]:
#Despoluindo o grafico
import folium
from folium.plugins import MarkerCluster
#Cria um mapa de acordo com a configuranções de geolocalização e zoom
map = folium.Map(location=[-22.7864889, -50.6786708], tiles='cartodbdark_matter', zoom_start=7)

mc = MarkerCluster()

for i in range(0, len(df3)): #ira criar varias marcações de acordo com a latitude e longitude do df3
  pop = str(df3.iloc[i]['nome'])
  pop = pop.encode("latin1")
  #folium.Marker([df3.iloc[i]['latitude'],df3.iloc[i]['longitude']],popup=pop).add_to(map)
  mc.add_child(folium.CircleMarker([df3.iloc[i]['latitude'],df3.iloc[i]['longitude']],radius = 10, popup=pop, color='blue', fill=True, fill_color='red'))

mc.add_to(map)
map

In [40]:
#3º Mapa
#Criando o mapa
import folium
from folium.plugins import MarkerCluster
#Cria um mapa de acordo com a configuranções de geolocalização e zoom
map = folium.Map(location = [-22.7864889, -50.678708], tiles = 'openStreetMap', zoom_start=7)
#Criando agrupamentos de forma manual
fg1 = folium.FeatureGroup(name = 'Municipio A')
fg2 = folium.FeatureGroup(name = 'Municipio B')
fg3 = folium.FeatureGroup(name = 'Municipio C')

for i in range (0,len(df3)): #ira criar varias marcações de acordo com a latitude e longitude do df3 de ordem alfabetica
  pop = str(df3.iloc[i]['nome'])
  pop = pop.encode("latin1") #Configurancao de idioma
  if i <= 200: #200 primeiros da ordem alfabetica    
    fg1.add_child(folium.CircleMarker([df3.iloc[i]['latitude'],df3.iloc[i]['longitude']],radius = 10, popup=pop, color='blue', fill=True, fill_color='red'))
  elif i <= 400: #400 primeiros na ordem alfabetica
    fg2.add_child(folium.CircleMarker([df3.iloc[i]['latitude'],df3.iloc[i]['longitude']],radius = 10, popup=pop, color='yellow', fill=True, fill_color='red'))
  else: # 401 em diante municios em ordem alfabetica
    fg3.add_child(folium.CircleMarker([df3.iloc[i]['latitude'],df3.iloc[i]['longitude']],radius = 10, popup=pop, color='green', fill=True, fill_color='red'))
map.add_child(fg1)
map.add_child(fg2)
map.add_child(fg3)
map.add_child(folium.LayerControl()) #permite eu escolher os municipios por filtro de camada

map  